In [1]:
import xarray as xr
import numpy as np
import pandas as pd

/home/byy/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/byy/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
era5_data = xr.open_dataset(r'./out.nc')
era5_data

<xarray.Dataset> Size: 2MB
Dimensions:     (valid_time: 98, longitude: 51, latitude: 26)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 784B 2019-10-29T23:00:00 ... 2019...
  * longitude   (longitude) float64 408B -80.5 -80.4 -80.3 ... -75.7 -75.6 -75.5
  * latitude    (latitude) float64 208B 45.0 44.9 44.8 44.7 ... 42.7 42.6 42.5
Data variables:
    u10         (valid_time, latitude, longitude) float32 520kB ...
    v10         (valid_time, latitude, longitude) float32 520kB ...
    sp          (valid_time, latitude, longitude) float32 520kB ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    history:                 Sun Jan 05 16:20:48 2025: cdo mergetime Oct-29.n...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

# Construct `fort.22` for NWS=6

In [25]:
import os
os.system("rm ./fort.22")
save_file = r'./fort.22'


lat_count = len(era5_data.latitude.values)
lon_count = len(era5_data.longitude.values)
lon = np.arange(-80.5, -75.5+0.1, 0.1)
lat = np.arange(45.0, 42.499, -0.1)
time_stamp = pd.date_range("2019-10-30 00", "2019-11-03 00", freq="1h")
time_count = len(time_stamp)
# time_count = 1 # for debugging

# FIXME: the efficiency may be too low 
with open(save_file, "ab") as f:
    for i in range(time_count):
        output_tot = np.empty((0,3))
        for j in range(lat_count):
            for k in range(lon_count):
                u10 = era5_data.u10.sel(valid_time=time_stamp[i], longitude=lon[k], latitude=lat[j],method='nearest')
                v10 = era5_data.v10.sel(valid_time=time_stamp[i], longitude=lon[k], latitude=lat[j],method='nearest')
                sp = era5_data.sp.sel(valid_time=time_stamp[i], longitude=lon[k], latitude=lat[j],method='nearest')
                # u10 = era5_data.u10.sel(valid_time=time_stamp[i+1], longitude=lon[k], latitude=lat[j])
                # v10 = era5_data.v10.sel(valid_time=time_stamp[i+1], longitude=lon[k], latitude=lat[j])
                # sp = era5_data.sp.sel(valid_time=time_stamp[i+1], longitude=lon[k], latitude=lat[j])
                output_arr = np.vstack((u10,v10,sp)).T
                output_tot = np.concatenate([output_tot, output_arr])
        np.savetxt(f, output_tot, fmt='%1.4f')
        print(str(i)+': '+str(time_stamp[i]))
        # print(output_tot) # for debugging

0: 2019-10-30 00:00:00
1: 2019-10-30 01:00:00
2: 2019-10-30 02:00:00
3: 2019-10-30 03:00:00
4: 2019-10-30 04:00:00
5: 2019-10-30 05:00:00
6: 2019-10-30 06:00:00
7: 2019-10-30 07:00:00
8: 2019-10-30 08:00:00
9: 2019-10-30 09:00:00
10: 2019-10-30 10:00:00
11: 2019-10-30 11:00:00
12: 2019-10-30 12:00:00
13: 2019-10-30 13:00:00
14: 2019-10-30 14:00:00
15: 2019-10-30 15:00:00
16: 2019-10-30 16:00:00
17: 2019-10-30 17:00:00
18: 2019-10-30 18:00:00
19: 2019-10-30 19:00:00
20: 2019-10-30 20:00:00
21: 2019-10-30 21:00:00
22: 2019-10-30 22:00:00
23: 2019-10-30 23:00:00
24: 2019-10-31 00:00:00
25: 2019-10-31 01:00:00
26: 2019-10-31 02:00:00
27: 2019-10-31 03:00:00
28: 2019-10-31 04:00:00
29: 2019-10-31 05:00:00
30: 2019-10-31 06:00:00
31: 2019-10-31 07:00:00
32: 2019-10-31 08:00:00
33: 2019-10-31 09:00:00
34: 2019-10-31 10:00:00
35: 2019-10-31 11:00:00
36: 2019-10-31 12:00:00
37: 2019-10-31 13:00:00
38: 2019-10-31 14:00:00
39: 2019-10-31 15:00:00
40: 2019-10-31 16:00:00
41: 2019-10-31 17:00:00
42

# Find `NWLAT, NWLON, WLATMAX, WLONMIN, WLATINC, WLONINC, WTIMINC` 

In [30]:
print("NWLAT: "+str(len(era5_data.latitude.values)))
print("NWLON: "+str(len(era5_data.longitude.values)))
print("WLATMAX: "+str(np.max(era5_data.latitude.values)))
print("WLONMIN: "+str(np.min(era5_data.longitude.values)))
print("WLATINC: "+str(np.abs(era5_data.latitude.values[0]-era5_data.latitude.values[1])))
print("WLONINC: "+str(np.abs(era5_data.longitude.values[0]-era5_data.longitude.values[1])))
print("WTIMINC: "+ "1h (3600s)")

NWLAT: 26
NWLON: 51
WLATMAX: 45.0
WLONMIN: -80.5
WLATINC: 0.10000000000000142
WLONINC: 0.09999999999999432
WTIMINC: 1h (3600s)
